<a href="https://colab.research.google.com/github/arturcgs/BootcampDataScienceAplicada3/blob/main/Aulas/M%C3%B3dulo_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

In [7]:
url = 'https://github.com/arturcgs/BootcampDataScienceAplicada3/raw/main/Bancos_de_Dados/dados_covid_sp.zip'

dados = pd.read_csv(url, sep=';', )

dados

,nome_munic,codigo_ibge,dia,mes,datahora,casos,casos_novos,casos_pc,casos_mm7d,obitos,obitos_novos,obitos_pc,obitos_mm7d,letalidade,nome_ra,cod_ra,nome_drs,cod_drs,pop,pop_60,area,map_leg,map_leg_s,latitude,longitude,semana_epidem
0,Adamantina,3500105,25,2,2020-02-25,0,0,"0,000000e+00","0,0000000",0,0,"0,000000e+00","0,0000000","0,0000000000",RA de Presidente Prudente,9,Marília,5,33894,7398,41199,0,8.0,"-21,6820","-51,0737",9
1,Adolfo,3500204,25,2,2020-02-25,0,0,"0,000000e+00","0,0000000",0,0,"0,000000e+00","0,0000000","0,0000000000",RA de São José do Rio Preto,14,São José do Rio Preto,15,3447,761,21106,0,8.0,"-21,2325","-49,6451",9
2,Aguaí,3500303,25,2,2020-02-25,0,0,"0,000000e+00","0,0000000",0,0,"0,000000e+00","0,0000000","0,0000000000",RA de Campinas,4,São João da Boa Vista,14,35608,5245,47455,0,8.0,"-22,0572","-46,9735",9
3,Águas da Prata,3500402,25,2,2020-02-25,0,0,"0,000000e+00","0,0000000",0,0,"0,000000e+00","0,0000000","0,0000000000",RA de Campinas,4,São João da Boa Vista,14,7797,1729,14267,0,8.0,"-21,9319","-46,7176",9
4,Águas de Lindóia,3500501,25,2,2020-02-25,0,0,"0,000000e+00","0,0000000",0,0,"0,000000e+00","0,0000000","0,0000000000",RA de Campinas,4,Campinas,3,18374,3275,6013,0,8.0,"-22,4733","-46,6314",9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286173,Ignorado,9999999,8,5,2021-05-08,820,116,"0,000000e+00","10,0000000",5,3,"0,000000e+00","-0,4285714","0,0060975610",NaN,0,NaN,0,0,0,0,NaN,NaN,"0,0000","0,0000",18
286174,Ignorado,9999999,9,5,2021-05-09,756,-64,"0,000000e+00","7,8571429",16,11,"0,000000e+00","1,0000000","0,0211640212",NaN,0,NaN,0,0,0,0,NaN,NaN,"0,0000","0,0000",19
286175,Ignorado,9999999,10,5,2021-05-10,703,-53,"0,000000e+00","3,8571429",9,-7,"0,000000e+00","0,2857143","0,0128022760",NaN,0,NaN,0,0,0,0,NaN,NaN,"0,0000","0,0000",19
286176,Ignorado,9999999,11,5,2021-05-11,744,41,"0,000000e+00","4,4285714",3,-6,"0,000000e+00","-0,1428571","0,0040322581",NaN,0,NaN,0,0,0,0,NaN,NaN,"0,0000","0,0000",19
